In [36]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

kaggle/input/digit-recognizer/test.csv
kaggle/input/digit-recognizer/train.csv


In [37]:
train_data = pd.read_csv("kaggle/input/digit-recognizer/train.csv")
train_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
test_data = pd.read_csv("kaggle/input/digit-recognizer/test.csv")
test_data.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class DigitDataset(Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target

    def __getitem__(self, item):
        x = torch.Tensor(self.data[item])
        y = torch.Tensor(self.target[item])
        return x, y

    def __len__(self):
        return len(self.data)

class DigitNet(nn.Module):
    def __init__(self):
        super(DigitNet, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 3)
        self.conv2 = nn.Conv1d(16, 32, 3)
        # (784 - 4) * 64
        self.fn1 = nn.Linear(780 * 32, 256)
        self.fn2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = x.view(-1, 780 * 32)
        x = self.fn1(x)
        x = nn.functional.relu(x)
        x = self.fn2(x)
        return x

In [40]:
X_train = train_data.to_numpy()[:,1:]
X_train = X_train.reshape((X_train.shape[0], 1, -1))
X_train = np.ascontiguousarray(X_train).astype(float)
y_train = np.zeros((X_train.shape[0], 10))
onehot = train_data.to_numpy()[:,0].T
y_train[np.arange(len(y_train)), onehot] = 1
y_train = np.ascontiguousarray(y_train).astype(float)
train_dataset = DigitDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [41]:
model = DigitNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(50):
    model.train()
    train_loss = 0.0
    for i, (x_batch, y_batch) in enumerate(train_loader):
        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss = nn.functional.cross_entropy(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x_batch.size(0)
    train_loss /= len(train_loader.dataset)

    model.eval()
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}')

KeyboardInterrupt: 

In [ ]:
X_test = test_data.to_numpy()
X_test = X_test.reshape((X_test.shape[0], 1, -1))
X_test = np.ascontiguousarray(X_test).astype(float)
with torch.no_grad():
    y_test = model(torch.Tensor(X_test))
pred = np.argmax(y_test, axis=1)
output = pd.DataFrame({'ImageId': np.arange(1, len(test_data) + 1), 'Label': pred})
print(output)